###  Quest
- NSC_BND_M20 사용 (목표변수:SICK_SYM1, 설명변수: 연속형변수 5개 사용)
- train_test_split() 이용
- classification_report(train, test set)사용, 평가 의견 첨부

#### **평가 의견**
- accuracy 0.024로 정확도가 굉장히 낮게 나옴 -> 좋지 않은 모델로 판단
- F1 score 0.024로 0에 가까운 수치 -> 좋지 않은 모델로 판단

#### 변수설정
1. 목표변수 : SICK_SYM1
2. 설명변수
    * MDCARE_DD_CNT(요양일수)
    * VSHSP_DD_CNT(입내원일수)
    * TOT_PRSC_DD_CNT(총처방일수)
    * EDC_SBA(심결본인부담금)
    * EDC_INSUR_BRDN_AMT(심결보험자부담금)

In [153]:
import pymongo as mg
import pandas as pd

In [154]:
client = mg.MongoClient(host='mongodb://localhost:27017')
database = client['db_NHIS']
collection = database['Join Collection NSC2_BND_M20']
cursor = collection.find({})
list_BNS_M20 = list(cursor)
df_BNS_M20 = pd.DataFrame(list_BNS_M20)
df_BNS_M20[:2]

,_id,RN_INDI,BTH_YYYY,DTH_YYYYMM,COD1,COD2,RN_KEY,RN_INST,MDCARE_STRT_DT,FORM_CD,...,OPRTN_YN,MDCARE_DD_CNT,VSHSP_DD_CNT,TOT_PRSC_DD_CNT,MCARE_RSLT_TYPE,EDC_ADD_RT,ED_RC_TOT_AMT,EDC_SBA,EDC_INSUR_BRDN_AMT,STD_YYYY
0,64e6c524f22b93c7fc16a128,294364,1921LE,200707.0,T08-T14,W00-W19,2002080298703,26452,20020830,3,...,0,1,1,3,5.0,0.15,10590,3000,7590,2002
1,64e6c524f22b93c7fc16a129,294364,1921LE,200707.0,T08-T14,W00-W19,2002090466395,63715,20020910,3,...,0,1,1,2,5.0,0.15,16520,4950,11570,2002


#### 전처리
- 3자리로 자르기
- F_, N_ 값들은 NULL 값으로 만들기 : 상병에 맞지 않는 값들이므로 자료를 없애주는 것

In [155]:
df_BNS_M20['SICK_SYM1'].value_counts()

J209     8244
I10      4629
I109     3275
F_       3203
N_       3177
         ... 
H700        1
E058        1
M5397       1
T18         1
P289        1
Name: SICK_SYM1, Length: 3414, dtype: int64

In [156]:
len('J209'), len('F_')

(4, 2)

In [157]:
'J209'[:3]
# str('J209')[:3]

'J20'

In [158]:
# sick_sym에 담기는 값들: J209, I10, I109...
def convertSICK_SYM(sick_sym) :
    if len(sick_sym) < 3 :
        return None
    else :
        return sick_sym[:3]
# return을 중간에 조건이 맞으면 쓰기도 함!

In [159]:
convertSICK_SYM('J209'), convertSICK_SYM('F_')

('J20', None)

In [160]:
# apply()적용해서 새로운 컬럼 만들기
df_BNS_M20['SICK_SYM1_3'] = df_BNS_M20['SICK_SYM1'].apply(convertSICK_SYM)

In [161]:
# null값 삭제
df_BNS_M20_dropna = df_BNS_M20.dropna()
df_BNS_M20_dropna.isnull().sum()

_id                   0
RN_INDI               0
BTH_YYYY              0
DTH_YYYYMM            0
COD1                  0
COD2                  0
RN_KEY                0
RN_INST               0
MDCARE_STRT_DT        0
FORM_CD               0
MCARE_SUBJ_CD         0
SICK_SYM1             0
OFIJ_TYPE             0
OPRTN_YN              0
MDCARE_DD_CNT         0
VSHSP_DD_CNT          0
TOT_PRSC_DD_CNT       0
MCARE_RSLT_TYPE       0
EDC_ADD_RT            0
ED_RC_TOT_AMT         0
EDC_SBA               0
EDC_INSUR_BRDN_AMT    0
STD_YYYY              0
SICK_SYM1_3           0
dtype: int64

In [162]:
df_BNS_M20_dropna[:2]

,_id,RN_INDI,BTH_YYYY,DTH_YYYYMM,COD1,COD2,RN_KEY,RN_INST,MDCARE_STRT_DT,FORM_CD,...,MDCARE_DD_CNT,VSHSP_DD_CNT,TOT_PRSC_DD_CNT,MCARE_RSLT_TYPE,EDC_ADD_RT,ED_RC_TOT_AMT,EDC_SBA,EDC_INSUR_BRDN_AMT,STD_YYYY,SICK_SYM1_3
0,64e6c524f22b93c7fc16a128,294364,1921LE,200707.0,T08-T14,W00-W19,2002080298703,26452,20020830,3,...,1,1,3,5.0,0.15,10590,3000,7590,2002,L02
3,64e6c524f22b93c7fc16a12b,294364,1921LE,200707.0,T08-T14,W00-W19,2002020488631,23506,20020208,3,...,1,1,3,5.0,0.15,12920,3000,9920,2002,K29


In [163]:
target_list = df_BNS_M20_dropna['SICK_SYM1_3'].value_counts().index.to_list()

In [164]:
target_list.index('L02'), target_list.index('J03')

(83, 6)

In [165]:
# SICK_SYM1_3 값을 넣었을 때 index값, 즉 어느 위치에 있는지 알고자 함
def applyIndexNumber(sick_sym_3):
    indexNumber = target_list.index(sick_sym_3)
    return indexNumber

In [166]:
df_BNS_M20_dropna['SICK_SYM1_3_target'] = df_BNS_M20_dropna['SICK_SYM1_3'].apply(applyIndexNumber)
df_BNS_M20_dropna[:2]

C:\Users\koreavc\AppData\Local\Temp\ipykernel_15936\2301855272.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_BNS_M20_dropna['SICK_SYM1_3_target'] = df_BNS_M20_dropna['SICK_SYM1_3'].apply(applyIndexNumber)


,_id,RN_INDI,BTH_YYYY,DTH_YYYYMM,COD1,COD2,RN_KEY,RN_INST,MDCARE_STRT_DT,FORM_CD,...,VSHSP_DD_CNT,TOT_PRSC_DD_CNT,MCARE_RSLT_TYPE,EDC_ADD_RT,ED_RC_TOT_AMT,EDC_SBA,EDC_INSUR_BRDN_AMT,STD_YYYY,SICK_SYM1_3,SICK_SYM1_3_target
0,64e6c524f22b93c7fc16a128,294364,1921LE,200707.0,T08-T14,W00-W19,2002080298703,26452,20020830,3,...,1,3,5.0,0.15,10590,3000,7590,2002,L02,83
3,64e6c524f22b93c7fc16a12b,294364,1921LE,200707.0,T08-T14,W00-W19,2002020488631,23506,20020208,3,...,1,3,5.0,0.15,12920,3000,9920,2002,K29,8


#### 정형화

In [167]:
from sklearn.model_selection import train_test_split
target = df_BNS_M20_dropna['SICK_SYM1_3_target']
features = df_BNS_M20_dropna[['MDCARE_DD_CNT', 'VSHSP_DD_CNT', 'TOT_PRSC_DD_CNT', 'EDC_SBA', 'EDC_INSUR_BRDN_AMT']]

In [168]:
features_train, features_test, target_train, target_test = train_test_split(features, target, random_state=111)
features_train.shape, target_train.shape, features_test.shape, target_test.shape

((621, 5), (621,), (208, 5), (208,))

#### 모델학습

In [169]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression()
model.fit(features_train, target_train)

C:\Users\koreavc\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

#### 예측

In [170]:
df_BNS_M20_dropna[0:5]

,_id,RN_INDI,BTH_YYYY,DTH_YYYYMM,COD1,COD2,RN_KEY,RN_INST,MDCARE_STRT_DT,FORM_CD,...,VSHSP_DD_CNT,TOT_PRSC_DD_CNT,MCARE_RSLT_TYPE,EDC_ADD_RT,ED_RC_TOT_AMT,EDC_SBA,EDC_INSUR_BRDN_AMT,STD_YYYY,SICK_SYM1_3,SICK_SYM1_3_target
0,64e6c524f22b93c7fc16a128,294364,1921LE,200707.0,T08-T14,W00-W19,2002080298703,26452,20020830,3,...,1,3,5.0,0.15,10590,3000,7590,2002,L02,83
3,64e6c524f22b93c7fc16a12b,294364,1921LE,200707.0,T08-T14,W00-W19,2002020488631,23506,20020208,3,...,1,3,5.0,0.15,12920,3000,9920,2002,K29,8
4,64e6c524f22b93c7fc16a12c,294364,1921LE,200707.0,T08-T14,W00-W19,2002060822342,23506,20020628,3,...,1,0,5.0,0.15,14230,3000,11230,2002,M54,59
5,64e6c524f22b93c7fc16a12d,294364,1921LE,200707.0,T08-T14,W00-W19,2002040690506,121328,20020422,3,...,2,5,5.0,0.15,22150,6000,16150,2002,J31,44
6,64e6c524f22b93c7fc16a12e,294364,1921LE,200707.0,T08-T14,W00-W19,2002070706300,79165,20020704,3,...,3,7,5.0,0.15,23510,9000,14510,2002,N30,20


In [171]:
model.predict(features_train[0:5])

array([53, 12, 67, 53, 53], dtype=int64)

#### 평가

In [172]:
target_train_predict = model.predict(features_train)
target_train_predict.shape

(621,)

In [173]:
from sklearn.metrics import accuracy_score

In [174]:
accuracy_score(target_train, target_train_predict)

0.024154589371980676

In [175]:
target_test_predict = model.predict(features_test)
target_test_predict.shape

(208,)

In [176]:
accuracy_score(target_test, target_test_predict)

0.02403846153846154

In [177]:
from sklearn.metrics import classification_report

In [178]:
print(classification_report(target_train, target_train_predict))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00        53
           1       0.07      0.08      0.08        49
           2       0.00      0.00      0.00        30
           3       0.00      0.00      0.00        29
           4       0.00      0.00      0.00        24
           5       0.00      0.00      0.00        28
           6       0.00      0.00      0.00        26
           7       0.00      0.00      0.00        18
           8       0.00      0.00      0.00        13
           9       0.00      0.00      0.00        14
          10       0.00      0.00      0.00        11
          11       0.00      0.00      0.00        13
          12       0.16      0.88      0.27         8
          13       0.00      0.00      0.00        10
          14       0.00      0.00      0.00         7
          15       0.00      0.00      0.00         8
          16       0.00      0.00      0.00         9
          17       0.00    

C:\Users\koreavc\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\koreavc\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\koreavc\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [179]:
print(classification_report(target_test, target_test_predict))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00        15
           1       0.00      0.00      0.00         7
           2       0.00      0.00      0.00        20
           3       0.00      0.00      0.00         6
           4       0.00      0.00      0.00         8
           5       0.00      0.00      0.00         4
           6       0.00      0.00      0.00         4
           7       0.00      0.00      0.00         6
           8       0.00      0.00      0.00        10
           9       0.00      0.00      0.00         5
          10       0.00      0.00      0.00         5
          11       0.00      0.00      0.00         2
          12       0.22      1.00      0.36         4
          13       0.00      0.00      0.00         2
          14       0.00      0.00      0.00         4
          15       0.00      0.00      0.00         3
          16       0.00      0.00      0.00         2
          17       0.00    

C:\Users\koreavc\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\koreavc\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\koreavc\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\koreavc\anaconda3\Lib\site-packages\sklearn\metr

#### 오차 행렬

In [180]:
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score

In [181]:
confusion_matrix(target_train, target_train_predict)

array([[0, 4, 0, ..., 0, 0, 0],
       [0, 4, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [182]:
precision_score(target_train, target_train_predict, average='micro')

0.024154589371980676

In [183]:
recall_score(target_train, target_train_predict, average='micro')

0.024154589371980676

In [184]:
f1_score(target_train, target_train_predict, average='micro')

0.024154589371980676